In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from mmnl import MMNL

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

In [5]:
qmc = MMNL(X,Y,75,3,method='QMC')
qmc.solver()

 final_simplex: (array([[ 1.43898751,  1.03571084, -1.05885721,  0.78078545,  0.25709845,
         1.0874467 ],
       [ 1.4389813 ,  1.03570542, -1.05885077,  0.78078068,  0.257171  ,
         1.08744476],
       [ 1.43897484,  1.03570558, -1.05885722,  0.78078975,  0.25718261,
         1.08744665],
       [ 1.43897646,  1.03570711, -1.05885908,  0.78078945,  0.25714921,
         1.08744694],
       [ 1.43898082,  1.03570874, -1.05885817,  0.78078311,  0.25701317,
         1.08744504],
       [ 1.43898804,  1.03570939, -1.05885308,  0.78079608,  0.2570333 ,
         1.08744844],
       [ 1.43897495,  1.03570647, -1.05885891,  0.78078701,  0.2571342 ,
         1.08745045]]), array([2815.25552749, 2815.25552749, 2815.25552749, 2815.25552749,
       2815.25552749, 2815.25552749, 2815.2555275 ]))
           fun: 2815.255527487398
       message: 'Optimization terminated successfully.'
          nfev: 943
           nit: 607
        status: 0
       success: True
             x: array([ 1.

 final_simplex: (array([[ 1.43898751,  1.03571084, -1.05885721,  0.78078545,  0.25709845,
         1.0874467 ],
       [ 1.4389813 ,  1.03570542, -1.05885077,  0.78078068,  0.257171  ,
         1.08744476],
       [ 1.43897484,  1.03570558, -1.05885722,  0.78078975,  0.25718261,
         1.08744665],
       [ 1.43897646,  1.03570711, -1.05885908,  0.78078945,  0.25714921,
         1.08744694],
       [ 1.43898082,  1.03570874, -1.05885817,  0.78078311,  0.25701317,
         1.08744504],
       [ 1.43898804,  1.03570939, -1.05885308,  0.78079608,  0.2570333 ,
         1.08744844],
       [ 1.43897495,  1.03570647, -1.05885891,  0.78078701,  0.2571342 ,
         1.08745045]]), array([2815.25552749, 2815.25552749, 2815.25552749, 2815.25552749,
       2815.25552749, 2815.25552749, 2815.2555275 ]))
           fun: 2815.255527487398
       message: 'Optimization terminated successfully.'
          nfev: 943
           nit: 607
        status: 0
       success: True
             x: array([ 1.

In [24]:
d = pd.read_csv('./data/catsup_trainformat.csv', delimiter=',')
d.head(10)

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1
5,0,1,1,0,0,4.3,1
6,1,1,2,0,0,5.2,1
7,0,1,3,0,0,4.4,1
8,0,1,0,0,0,4.6,2
9,0,1,1,0,1,2.5,2


In [25]:
X = d.values[:,1:-1]
X.shape

(11192, 5)

In [34]:
def summarize_data(model): 
    #model should be in method(draws) format
    result = []
    for file in os.listdir('.\\resultaten\\%s'%(model)):
        if file.endswith('%s_results2.p'%(model)):
            infile = open(os.path.join('.\\resultaten\\%s'%(model), file), 'rb')
            opt_list = pickle.load(infile)
            [result.append(i) for i in opt_list]
    
    pickle.dump( result, open( "./resultaten/%s/%s_all_results.p" %(model,model), "wb" ) )
    return result      
        

In [38]:
def probs(X,model,theta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    method = model.split('(')[0]
    R = int(model.split('(')[1][:-1])
    print(R)
    if method == 'QMC':
        delta = QMC(300,3,R)
    elif method == 'SMC':
        delta = np.random.randn(300,3,R)
    
    beta = theta[:3][:, None] + delta * np.abs(theta[3:][:, None])
    try:
        assert(X.shape == (11192, 5) and beta.shape == (300,3,R))
        
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192,R))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i,:] = beta[id-1,:,:]
        
    P = np.zeros((11192,R))
    for r in range(R):
        P[:,r] = np.exp((X[:,2:]@ beta_choice[:,:,r])[:,0])
        
    try:
        assert(P.shape == (11192,R) or P.shape == (R,11192))
    except AssertionError:
        raise AssertionError('Product van X en beta gaat niet goed, P.shape is nu %g' % (P.shape))
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = np.zeros(11192,)
    for i in range(0,11192, 4):
        sum = np.sum(P[i:i+4,:],axis=0)
        Y[i:i+4] = np.mean(P[i:i+4]/sum,axis=1)
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
    return Y
# Y_true2 = probs(X,'QMC(1000)',np.array([1.5,1,-1.1,0.8,0.1,1.2]))

In [31]:
pickle.dump( Y_true2, open( "./resultaten/Y_true2_QMC(1000).p", "wb" ) )


In [6]:
infile = open('./resultaten/QMC(75)/QMC(75)_all_results.p', 'rb')
qmc75 = pickle.load(infile)

In [7]:
print(len(qmc75))

100


In [32]:
theta_true = np.array([1.5,1,-1.1,0.8,0.1,1.2])

In [33]:
def get_results(model_data,method,Y_true):
    #model data should be in list format containing OptimizeResult types
    theta_true = np.array([1.5,1,-1.1,0.8,0.1,1.2])
    results ={}
    theta_est = np.zeros((6,))
    mape = 0
    for run in model_data:
        theta_est += run.x/len(model_data)
#     P = probs(X,method,theta_est)
#     mape_choice = 100*np.sum(np.abs((Y_true-P)/Y_true))/11192
#     rmse_choice = np.sqrt(np.sum((Y_true-P)**2)/11192)
    mape_par = 100*np.sum(np.abs((theta_true-theta_est)/theta_true))/6
    rmse_par = np.sqrt(np.sum((theta_true-theta_est)**2)/6)

    results = { 'method': method,
                'theta': theta_est,
#                 'choice': {'mape': mape_choice,
#                            'rmse': rmse_choice,
#                           },
               'Parameter': {'mape': mape_par,
                             'rmse': rmse_par
                           }
            }
    return results

Get true probabilities based on big QMC model

In [35]:
qmc25 = summarize_data('QMC(25)')

In [37]:
len(qmc25)
resqmc25 = get_results(qmc25,'QMC(25)',Y_true)
print(resqmc25)

{'method': 'QMC(25)', 'theta': array([ 0.96361293,  0.8029465 , -1.11864207,  0.70108456,  0.46697882,
        0.03907107]), 'Parameter': {'mape': 88.8744240008519, 'rmse': 0.550621223547447}}


In [14]:
infile = open('./resultaten/SMC(500)/0-100_SMC(500)_results.p', 'rb')
smc500 = pickle.load(infile)
len(smc500)

100

In [15]:
ressmc500 = get_results(smc500, 'SMC(500)', Y_true)
ressmc500

500


{'method': 'SMC(500)',
 'theta': array([ 1.26975361,  0.96992687, -1.01877069,  0.17900136,  0.00968623,
         0.07404637]),
 'choice': {'mape': 25.517837754510417, 'rmse': 0.05342257395388834},
 'Parameter': {'mape': 47.91827122643141, 'rmse': 0.5357367916390605}}

In [16]:
smc1000 = summarize_data('SMC(1000)')
len(smc1000)

100

In [21]:
ressmc1000 = get_results(summarize_data('SMC(1000)'), 'SMC(1000)', Y_true)

In [20]:
ressmc1000

{'method': 'SMC(1000)',
 'theta': array([ 1.48092315,  0.81189818, -1.09673566,  0.03284022,  0.16863493,
         0.08168019]),
 'Parameter': {'mape': 46.35032459069284, 'rmse': 0.5597083997209648}}

In [19]:
len(smc1000)

210

In [13]:
# bqmc8 = summarize_data('BQMC(8)')
len(bqmc8)

256

In [17]:
resbqmc8 = get_results(bqmc8,'BQMC(8)', Y_true)
resbqmc8

{'method': 'BQMC(8)',
 'theta': array([ 0.93072305,  0.5005806 , -1.19101155,  0.61505522,  0.5443808 ,
         0.20131094]),
 'Parameter': {'mape': 107.81508341898723, 'rmse': 0.5493690005579851}}

In [11]:
model = 'SMC(500)'
for file in os.listdir('.\\resultaten\\%s'%(model)):
    if file.endswith('%s_results.p'%(model)):
        infile = open(os.path.join('.\\resultaten\\%s'%(model), file), 'rb')
        opt_list = pickle.load(infile)
        print(len(opt_list),file)

100 0-100_SMC(500)_results.p
